In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer        
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

steps of pipeline..

train_test_split -->missing values --> OHE --> Scaling --> Feature Scaling --> Decision Tree

In [4]:

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                    random_state=42)

In [5]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [6]:
y_train.sample(4)

61     1
220    1
29     0
237    1
Name: Survived, dtype: int64

In [7]:
#imputation transformers
trf1 = ColumnTransformer([
    ('imputer_age', SimpleImputer(), [2]), 
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')                       

In [8]:
#ONE HOT ENCODING
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), [1,6])
], remainder='drop')

In [9]:
#scaling
trf3 = ColumnTransformer([
    ('minmax_scaler', MinMaxScaler(),slice(0,10))
], remainder='passthrough')

In [10]:
#feature selection
trf4 = SelectKBest(score_func=chi2, k=5)

In [11]:
#training the model
trf5 = DecisionTreeClassifier()

create pipeline

In [12]:
pipe =Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

OR ALTERNATE SYNTAX

In [13]:
#pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)


In [14]:
from sklearn import set_config
set_config(enable_metadata_routing=True,display='diagram')

In [15]:
pipe.fit(x_train, y_train)

,steps,"[('trf1', ...), ('trf2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('imputer_age', ...), ('impute_embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


explore the pipelines


In [16]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [17]:
pipe.named_steps['trf2'].transformers_[1][2]

[0, 2, 3, 4, 5]

In [18]:
pipe.named_steps['trf3'].transformers_

[('minmax_scaler', MinMaxScaler(), slice(0, 10, None)),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   12

In [19]:
y_pred = pipe.predict(x_test)
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6256983240223464


EXPORTING THE PIPELINE


In [20]:
import pickle
pickle.dump(pipe, open('titanic_model.pkl', 'wb'))

CROSS VALIDATION USING PIPELINE


In [21]:
#with cross validation we use this for better evaluation basically it uses different splits of data for training and testing
#5 baar(cv=5) model ko train test split krta hai or phir unka average nikalta hai 
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe,x_train, y_train , cv=5).mean()
print("Cross-validation scores mean:", scores)

Cross-validation scores mean: 0.6419186447355462


GRIDSEARCH USING PIPELINES

In [22]:
params =(
    'trf5__max_depth', [3,5,7,9,None],)

In [23]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(pipe, dict([params]), cv=5, scoring='accuracy', n_jobs=-1)
gs.fit(x_train, y_train)


,estimator,Pipeline(step...lassifier())])
,param_grid,"{'trf5__max_depth': [3, 5, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('imputer_age', ...), ('impute_embarked', ...)]"


In [24]:
gs.best_params_

{'trf5__max_depth': 3}

In [25]:
gs.best_score_

0.6433270954397715